In [101]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
from sklearn import metrics
import os
import yaml
import nltk
import re
from nltk.corpus import stopwords
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

In [61]:
train=pd.read_csv("train.csv")
samp=pd.read_csv("sample_submission.csv")
test=pd.read_csv("test.csv",encoding="ISO-8859-1")

In [4]:
train.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [5]:
def processQues(chat):
    chat = chat.lower()
    chat = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',chat)
    chat = re.sub('@[^\s]+','',chat)
    chat = re.sub('[\s]+', ' ', chat)
    chat = re.sub('\[\d+\]+', ' ', chat)
    chat = re.sub(r'#([^\s]+)', r'\1', chat)
    chat = re.sub(r'[\!:\?\-\'\"\\/]', r'', chat)
    chat = chat.strip('\'"')
    return chat

In [6]:
train[5:8]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0


In [7]:
stop_words = set(stopwords.words('english'))

In [8]:
train.shape

(159571, 8)

In [9]:
def CleanText(comment):
    comment=processQues(comment)
    comment = nltk.word_tokenize(comment)
    w = [wd for wd in comment if wd not in stop_words]
    return " ".join(w)

In [10]:
train["comment_text"]=train["comment_text"].apply(CleanText)
test["comment_text"]=test["comment_text"].apply(CleanText)

In [44]:
test.shape

(63978, 2)

In [45]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63978 entries, 0 to 63977
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            63978 non-null  object
 1   comment_text  63978 non-null  object
dtypes: object(2)
memory usage: 999.8+ KB


In [47]:
test.tail(10)

,id,comment_text
63968,fff69311f306df44,balancing two approaches psychiatry ( biologic...
63969,fff7159b3ee95618,"== name mentioned == hi , thought letting know..."
63970,fff718ffe5f05559,ive discovered yet another list list composers...
63971,fff83b80284d8440,consensus ruining wikipedia think ignored .
63972,fff8f521a7dbcd47,shut mexican border withought looking bad .
63973,fff8f64043129fa2,"jerome , see never got around thisâ¦ iâm su..."
63974,fff9d70fe0722906,==lucky bastard== heh famous . kida envy . con...
63975,fffa8a11c4378854,==shame all== want speak gays romanians ...
63976,fffac2a094c8e0e2,mel gibson nazi bitch makes shitty movies . mu...
63977,fffb5451268fb5ba,"== unicorn lair discovery == supposedly , unic..."


In [38]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww matches background colour im seemingly st...,0,0,0,0,0,0
2,000113f07ec002fd,"hey man , im really trying edit war . guy cons...",0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real suggestions improvement wondere...,0,0,0,0,0,0
4,0001d958c54c6e35,", sir , hero . chance remember page thats",0,0,0,0,0,0


In [11]:
list_sentences_train = train["comment_text"].values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].values

## Here we will use GloVe word embedding for text encoding.

In [12]:
EMBEDDING_FILE='glove.6B.100d.txt'

In [13]:
list_sentences_train[0]

'explanation edits made username hardcore metallica fan reverted werent vandalisms , closure gas voted new york dolls fac . please dont remove template talk page since im retired now.89.205.38.27'

In [14]:
#tokenizer.texts_to_sequences(list_sentences_train[0])

In [15]:
embed_size = 100 # how big is each word vector
max_features = 40000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [16]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))

list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [50]:
X_train[-10:]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,    63,     6,   225,   416,    28, 21753,  1982,   180,
         1386,   116,  1528,  9611,  2078,   245,   400,  1528,    63,
          243,  2546,    46,   650,   112,  5057,   250,  4374,   794,
         8842,    25,   147,  7797,    94,   108,  1802,  2078,   805,
         4356,  1252,   191,   109,    63,  1252,  4496,   128,   234,
           26,   572,  1341,    11,  2998,     9,  1802,  2078,    49,
         4356,    48,   143,  1085,   319,  7981,   100,   191,   265,
         1191,   847,   109, 23316,     2,   572,   461,     2,   571,
          229, 35639,  1848,   319,   100,  5018,   169,  1252,   245,
         3087,   243,   391,    15,    70,    29,   388,  1085,  1666,
          572],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,  5815,  6243,
        34107,   880,    25,    30,   232,     1, 11768,  322

In [17]:
word_index = tokenizer.word_index

In [44]:
word_index

{'article': 1,
 'page': 2,
 'wikipedia': 3,
 'talk': 4,
 'please': 5,
 'would': 6,
 'one': 7,
 'like': 8,
 'dont': 9,
 'see': 10,
 'also': 11,
 'think': 12,
 'im': 13,
 'know': 14,
 'people': 15,
 'edit': 16,
 'articles': 17,
 'use': 18,
 'may': 19,
 'time': 20,
 'thanks': 21,
 'even': 22,
 'get': 23,
 'make': 24,
 'good': 25,
 'information': 26,
 'well': 27,
 'could': 28,
 'want': 29,
 'sources': 30,
 'way': 31,
 'name': 32,
 'deletion': 33,
 'pages': 34,
 'first': 35,
 'help': 36,
 'new': 37,
 'source': 38,
 'editing': 39,
 'go': 40,
 'need': 41,
 'say': 42,
 'section': 43,
 'edits': 44,
 'user': 45,
 'thank': 46,
 'made': 47,
 'many': 48,
 'much': 49,
 'really': 50,
 'used': 51,
 'fuck': 52,
 'deleted': 53,
 'discussion': 54,
 'find': 55,
 'ive': 56,
 'work': 57,
 'since': 58,
 'point': 59,
 'right': 60,
 'add': 61,
 'look': 62,
 'read': 63,
 'image': 64,
 'take': 65,
 'still': 66,
 'someone': 67,
 'back': 68,
 'two': 69,
 'fact': 70,
 'link': 71,
 'said': 72,
 'something': 73,
 'yo

In [45]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

In [18]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf8"))

In [21]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

C:\Users\pranj\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


(0.004451992, 0.4081574)

In [49]:
word_index

{'article': 1,
 'page': 2,
 'wikipedia': 3,
 'talk': 4,
 'please': 5,
 'would': 6,
 'one': 7,
 'like': 8,
 'dont': 9,
 'see': 10,
 'also': 11,
 'think': 12,
 'im': 13,
 'know': 14,
 'people': 15,
 'edit': 16,
 'articles': 17,
 'use': 18,
 'may': 19,
 'time': 20,
 'thanks': 21,
 'even': 22,
 'get': 23,
 'make': 24,
 'good': 25,
 'information': 26,
 'well': 27,
 'could': 28,
 'want': 29,
 'sources': 30,
 'way': 31,
 'name': 32,
 'deletion': 33,
 'pages': 34,
 'first': 35,
 'help': 36,
 'new': 37,
 'source': 38,
 'editing': 39,
 'go': 40,
 'need': 41,
 'say': 42,
 'section': 43,
 'edits': 44,
 'user': 45,
 'thank': 46,
 'made': 47,
 'many': 48,
 'much': 49,
 'really': 50,
 'used': 51,
 'fuck': 52,
 'deleted': 53,
 'discussion': 54,
 'find': 55,
 'ive': 56,
 'work': 57,
 'since': 58,
 'point': 59,
 'right': 60,
 'add': 61,
 'look': 62,
 'read': 63,
 'image': 64,
 'take': 65,
 'still': 66,
 'someone': 67,
 'back': 68,
 'two': 69,
 'fact': 70,
 'link': 71,
 'said': 72,
 'something': 73,
 'yo

In [50]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [52]:
embedding_matrix[3],len(embedding_matrix[0])

(array([-0.2536    ,  0.21885   ,  0.53349   , -0.52443999,  0.53654999,
         0.62448001, -0.12988999, -0.83825999,  0.89195001,  0.033484  ,
         0.42016   ,  0.44988   ,  0.094579  , -0.92764002, -0.48991001,
         0.75895   ,  0.48857999, -0.57347   , -0.75297999,  0.53346002,
        -0.72722   ,  0.41163999,  0.049068  ,  0.59324002,  0.028872  ,
        -1.44690001,  0.072449  , -0.051847  ,  0.36256999,  0.1662    ,
         0.022671  ,  1.26300001, -0.634     , -0.72939003,  0.29486001,
         0.41602999, -0.40253001, -0.21218   , -0.71228999, -0.04464   ,
        -0.80034   ,  0.83279002, -0.24826001,  0.61856002, -0.26475999,
         0.38703001, -0.026548  , -0.85908002,  0.34218001,  0.28380999,
         0.79504001,  0.78182   , -0.81676   , -0.023553  , -1.42820001,
        -0.065081  , -0.36142999, -0.38418001,  0.49507999, -0.079691  ,
        -0.21495   ,  0.3556    , -0.55287999, -0.14088   ,  1.36839998,
         0.29986   , -0.051735  , -0.27048999,  0.6

In [ ]:
embedding_matrix.shape

In [53]:
def enable_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
enable_gpu()

In [ ]:
#inp = Input(shape=(maxlen,))
#x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
#x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
#x = GlobalMaxPool1D()(x)
#x = Dense(50, activation="relu")(x)
#x = Dropout(0.1)(x)
#x = Dense(6, activation="softmax")(x)
#model = models.Model(inputs=inp, outputs=x)
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model=Sequential()
model.add(Embedding(max_features,embed_size, weights=[embedding_matrix]))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embed_size)))
model.add(Dropout(0.2))
model.add(Dense(6, activation="softmax"))
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

In [20]:
model.fit(X_train, y, batch_size=32, epochs=10, validation_split=0.2, verbose=2)

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 - 196s - loss: 0.3814 - accuracy: 0.4528 - val_loss: 0.3455 - val_accuracy: 0.9474
Epoch 2/10
127656/127656 - 205s - loss: 0.3717 - accuracy: 0.4383 - val_loss: 0.3401 - val_accuracy: 0.9094
Epoch 3/10
127656/127656 - 217s - loss: 0.3702 - accuracy: 0.4132 - val_loss: 0.3732 - val_accuracy: 0.9737
Epoch 4/10
127656/127656 - 220s - loss: 0.4036 - accuracy: 0.4166 - val_loss: 0.4237 - val_accuracy: 0.9676
Epoch 5/10
127656/127656 - 218s - loss: 0.3997 - accuracy: 0.4098 - val_loss: 0.3370 - val_accuracy: 0.9463
Epoch 6/10
127656/127656 - 215s - loss: 0.3631 - accuracy: 0.4215 - val_loss: 0.3436 - val_accuracy: 0.9728
Epoch 7/10
127656/127656 - 219s - loss: 0.3629 - accuracy: 0.4312 - val_loss: 0.3632 - val_accuracy: 0.8902
Epoch 8/10
127656/127656 - 216s - loss: 0.3630 - accuracy: 0.4295 - val_loss: 0.3320 - val_accuracy: 0.9847
Epoch 9/10
127656/127656 - 218s - loss: 0.3569 - accuracy: 0.4412 - val_loss: 0.3388 

In [21]:
model.save("model_final_Toxic_new_with_gensim.h5")
print("Saved model to disk")

Saved model to disk


In [51]:
#Load Model
from tensorflow.keras.models import load_model
model_loaded = load_model('model_final_Toxic_new_with_gensim.h5')

In [52]:
print(model_loaded.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         4000000   
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1206      
Total params: 4,162,006
Trainable params: 4,162,006
Non-trainable params: 0
_________________________________________________________________
None


In [89]:
y_test = model_loaded.predict([X_test], batch_size=1024, verbose=1)
y_train_pred=model_loaded.predict([X_train], batch_size=1024, verbose=1)

156/156 [==============================] - 7s 44ms/step


In [54]:
y_test[-10:]

array([[0.27449477, 0.13260743, 0.17757362, 0.11811639, 0.17110473,
        0.12610313],
       [0.27461737, 0.13292603, 0.17713325, 0.11840017, 0.17076376,
        0.12615947],
       [0.2746133 , 0.13285917, 0.17713061, 0.11845624, 0.17084463,
        0.12609601],
       [0.2746512 , 0.13294241, 0.17715506, 0.11841476, 0.1707848 ,
        0.12605186],
       [0.30932572, 0.0057211 , 0.3581677 , 0.00711394, 0.300615  ,
        0.01905649],
       [0.2745945 , 0.13291495, 0.17711847, 0.1183903 , 0.17083292,
        0.12614894],
       [0.5065503 , 0.00934143, 0.25063816, 0.00855963, 0.21292166,
        0.01198878],
       [0.2693983 , 0.1321303 , 0.17658946, 0.1153587 , 0.17745382,
        0.1290695 ],
       [0.5013046 , 0.00849585, 0.25404882, 0.00685667, 0.21825652,
        0.01103747],
       [0.27462617, 0.13462864, 0.16286959, 0.11106741, 0.18663976,
        0.13016836]], dtype=float32)

## Threshould value based on class 20%

In [90]:
y_test_df=pd.DataFrame(y_test, columns=list_classes)

In [91]:
y_train_pred_df=pd.DataFrame(y_train_pred, columns=list_classes)

In [93]:
def thres(val):
    if(val>=0.20):
        return 1
    else:
        return 0

In [94]:
y_train_pred_df["toxic"]=y_train_pred_df["toxic"].apply(thres)
y_train_pred_df["severe_toxic"]=y_train_pred_df["severe_toxic"].apply(thres)
y_train_pred_df["obscene"]=y_train_pred_df["obscene"].apply(thres)
y_train_pred_df["threat"]=y_train_pred_df["threat"].apply(thres)
y_train_pred_df["insult"]=y_train_pred_df["insult"].apply(thres)
y_train_pred_df["identity_hate"]=y_train_pred_df["identity_hate"].apply(thres)

In [98]:
test[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]]=y_test_df

In [99]:
test["toxic"]=test["toxic"].apply(thres)
test["severe_toxic"]=test["severe_toxic"].apply(thres)
test["obscene"]=test["obscene"].apply(thres)
test["threat"]=test["threat"].apply(thres)
test["insult"]=test["insult"].apply(thres)
test["identity_hate"]=test["identity_hate"].apply(thres)

In [100]:
test.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,thank understanding . think highly would rever...,1,0,0,0,0,0
1,000247e83dcc1211,dear god site horrible .,1,0,0,0,1,0
2,0002f87b16116a7f,somebody invariably try add religion really me...,1,0,0,0,0,0
3,0003e1cccfd5a40a,says right type . type institution needed case...,1,0,0,0,0,0
4,00059ace3e3e9a53,"== adding new product list , make sure relevan...",1,0,0,0,0,0
5,000663aff0fffc80,one 1897,1,0,0,0,0,0
6,000689dd34e20979,== reason banning throwing == article needs se...,1,0,0,0,0,0
7,000844b52dee5f3f,|blocked ] ] editing wikipedia . |,1,0,0,0,0,0
8,00091c35fa9d0465,"== arabs committing genocide iraq , protests e...",1,0,0,0,1,0
9,000968ce11f5ee34,"please stop . continue vandalize wikipedia , h...",1,0,0,0,0,0


In [106]:
print(metrics.classification_report(train[list_classes],y_train_pred_df))

              precision    recall  f1-score   support

           0       0.09      0.97      0.17     15294
           1       0.00      0.00      0.00      1595
           2       0.70      0.87      0.77      8449
           3       0.01      0.01      0.01       478
           4       0.54      0.84      0.66      7877
           5       0.02      0.00      0.01      1405

   micro avg       0.16      0.82      0.26     35098
   macro avg       0.23      0.45      0.27     35098
weighted avg       0.33      0.82      0.41     35098
 samples avg       0.08      0.09      0.08     35098



## Here we can see what kind to classes belongs to the comments!

Submitted By, Pranjal Saxena <a>https://www.linkedin.com/in/pranjalai/ </a> <br>
pranjal.saxena2012@gmail.com